# Project: Identifying Spammers on Social Networks


# Classification Problem

In this notebook the main focus is classification of whether a user is a spammer or not. 

In [1]:
#Import Libraries
import tensorflow as tf 
import pandas as pd
import numpy as np
import scipy.sparse as sp
import csv 

## Data Preprocessing

We work with the whole dataset. Basically we read the original user dataset and save it. After that we start reading the original relations dataset line by line from the "relations.csv" file and we create 2 dictionaries.

One of the dictonaries contains the sum of each relation type with respect to each node, where the node was the source, and the other dictionary contains the sum of each relation type with respect to each node, where the node was the destination. In simple words for each node and for each relation type of that node, we get the in-degree and out-degree sum. 

In [3]:
data_folder = "./data/"
users = pd.read_csv(data_folder+'usersdata.csv',delimiter='\t', names=['User Id','Gender', 'Time', 'Age Range','Spammer Label'])

In [4]:
#Define dictionaries
full_src_data = {}
full_dst_data = {}

#### Note: 
The cell below, where we read line by line of the relation.csv file will not be executed in the final classficiation notebook, since it takes a lot of time to finish and we already executed it once and saved the results that we needed. For execution of the cell, all needs to be done is to have a "data" folder containing the original "relations.csv" dataset. 

In [ ]:
#Read line by line from the relation.csv and update the corresponding node-relation_type with +1.
with open("./data/"+"relations.csv", "rt") as f_in:
    reader             = csv.reader(f_in, delimiter="\t")
    
    for i, line in enumerate(reader):    
        node_1             = int(line[2])  
        node_2             = int(line[3])
        
        key1_src = str(node_1) + "_"  + line[4]
        key2_dst = str(node_2) + "_"  + line[4]
        if key1_src in full_src_data:
            full_src_data[key1_src] += 1
        else:
            for i in range(1,8):
                key1_src = str(node_1) + "_" + str(i)
                full_src_data[key1_src] = 0
            key1_src = str(node_1) + "_" + line[4]
            full_src_data[key1_src] += 1
        
        if key2_dst in full_dst_data:
            full_dst_data[key2_dst] += 1
        else:
            for i in range(1,8):
                key2_dst = str(node_2) + "_" + str(i)
                full_dst_data[key2_dst] = 0
            key2_dst = str(node_2) + "_" + line[4]
            full_dst_data[key2_dst] +=1
        if i%1000 is 0:
            print("\r"+str(i) +" relations processed", sep=' ', end='', flush=True)

Once the dictionaries are set, we are using the data that is in them, for construting new features. Will basically add 14 new features to our normal features. Each feature is going to be either where the node was source or a destination with respect to each relation type. 

In [6]:
#Create np for source nodes and destination nodes, such that we can save the results from the dictionaries
rel_src = np.zeros((users.shape[0],7))
rel_dst = np.zeros((users.shape[0],7))

In [7]:
#Creates an np matrix that will contain the features
def create_features_src_dst(data, mat):
    for key, value in data.items():
        k = key.split("_")
        node = (int(k[0]) - 1)
        rel = (int(k[1]) - 1)
        if(node != (full_users.shape[0])):
            mat[node][rel] = value 
    return mat

In [8]:
#Execute and create the features
rel_src = create_features_src_dst(full_src_data, rel_src)
rel_dst = create_features_src_dst(full_dst_data, rel_dst)

From the numpy matrices that we have created which contain the in-degree and out-degree with respect to each node and to each relation, we create 2 new dataframes. Later we merge these two dataframes into one.

Later we merge this newly formed dataframe with the original users dataframe, such that now we have enriched the dataset with 14 new features. 

In [9]:
#Create Pandas Dataframes out of the numpy matrices
df = pd.DataFrame(rel_src, columns=["1_src", "2_src", "3_src", "4_src", "5_src", "6_src", "7_src"])
df1 = pd.DataFrame(rel_dst, columns=["1_dst", "2_dst", "3_dst", "4_dst", "5_dst", "6_dst", "7_dst"])

In [10]:
#Merge the 2 dataframes such that we get one complete dataframe
df = pd.merge(df1, df, left_index=True, right_index=True)

In [11]:
#Merge the newly formed dataframe of relations based on whether the node is source or destination with the users table
#This would form a new database where every column is considered as a feature, without the "Spammer Label" Column.
new_users = pd.merge(users, df, left_index=True, right_index=True)

#### Note:

Since the cell where we read through the whole relation.csv is not executed, in the next cell we will just read a csv file which contains exactly all the stuff that the whole data preprocessing step does.

In [13]:
#Read the already formed data.
#If all of the above cells have been executed this cell does not need to be executed. 
new_users = pd.read_csv(data_folder+"merged_users_relations.csv", sep=",")
new_users.drop(columns=["Unnamed: 0"], inplace=True)
new_users.head()


,User Id,Gender,Time,Age Range,Spammer Label,1_dst,2_dst,3_dst,4_dst,5_dst,6_dst,7_dst,1_src,2_src,3_src,4_src,5_src,6_src,7_src
0,1,M,0.9000,30,0,0.0,0.0,24.0,39.0,21.0,112.0,0.0,0.0,0.0,154.0,154.0,225.0,98.0,0.0
1,2,F,1.0000,20,0,1.0,0.0,28.0,32.0,63.0,117.0,0.0,0.0,0.0,0.0,84.0,21.0,308.0,0.0
2,3,M,0.1375,30,0,0.0,5.0,11.0,2.0,6.0,5.0,0.0,0.0,28.0,168.0,42.0,0.0,0.0,0.0
3,4,M,0.3875,20,0,0.0,0.0,4.0,3.0,9.0,7.0,0.0,0.0,0.0,7.0,42.0,42.0,0.0,0.0
4,5,M,0.0125,20,0,0.0,0.0,1.0,5.0,6.0,10.0,0.0,0.0,0.0,28.0,42.0,35.0,7.0,0.0


## Set up of the training and testing dataset

In this section we split the training and testing data such that we can fit it to our Artificial Neural Network. We split the dataframe in features and labels. 

1. The only feature that is categorical is the "Gender" feature, thus we encode properly as a number. 
2. Split the dataset into 80% training and 20% testing. 
3. Standardize both the training and testing data. 

In [14]:
#Import libraries for setting the ANN and the training and testing data set
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [16]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [17]:
# Extract features and labels
x = new_users[["Gender", "Time", "Age Range", "1_src", "2_src", "3_src", "4_src", "5_src", "6_src", "7_src", "1_dst", "2_dst", "3_dst", "4_dst", "5_dst", "6_dst", "7_dst"]].copy()
y = new_users["Spammer Label"]

In [18]:
# Set them as numpy arrays
x = x.values
y = y.values

In [20]:
# Since the "Gender" feature is categorical we transform it to dummy variable.
encoder = LabelEncoder()
x[:,0] = encoder.fit_transform(x[:,0])

In [21]:
# Transform all features to float type.
x = x.astype(float)

In [22]:
# Encode the spammer label. 
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

In [23]:
#Split to 80% training data and 20% to testing data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 0)

In [24]:
#Standardize the data
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

## Creation of the Artificial Neural Network

We create an Artificial Neural Network, which has 15,503 parameters. It contains 3 hidden layers. The neural network uses the adam optimizer and optimizes with respect to the f1_loss function. 

The f1_loss function is a loss function created by us. Since the data is pretty skewed, aiming for accuracy would not be good. This is due to the fact that there are like 7% spammers in the whole data set and 93% of non-spammers, thus if we just set to classify everybody as non-spammer, we would get a 93% accuracy. Therefore we optimize for F1 score.

Once we create the model, we run it with 200 epochs and with a batch_size of 1000000.
With this we manage to predict 56-58% of the spammers in testing set and 95-96% of the non-spammers in the testing set. 

In [35]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(34, input_dim=17, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(34, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(100, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(100, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    # Compile model
    model.compile(loss=f1_loss, optimizer='adam', metrics=["binary_accuracy",'accuracy',f1])
    return model

In [36]:
from keras import backend as K

#We create our own metrics, basically the f1_score so we can see how well are we doing
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)


#We create our own loss function, the f1_loss which will optimize to get a better f1_score actually being linear
def f1_loss(y_true, y_pred):

    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [34]:
#Create the model and Show the summary of the ANN
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 34)                612       
_________________________________________________________________
dense_7 (Dense)              (None, 34)                1190      
_________________________________________________________________
dense_8 (Dense)              (None, 100)               3500      
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 15,503
Trainable params: 15,503
Non-trainable params: 0
_________________________________________________________________


In [42]:
#Train the Model 
#The ANN was trained with a GPU RTX 2080
model.fit(x_train, y_train, epochs = 200, batch_size = 1000000)

Epoch 1/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5388 - binary_accuracy: 0.9452 - acc: 0.9452 - f1: 0.4621
Epoch 2/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5389 - binary_accuracy: 0.9452 - acc: 0.9452 - f1: 0.4620
Epoch 3/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5389 - binary_accuracy: 0.9451 - acc: 0.9451 - f1: 0.4620
Epoch 4/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5391 - binary_accuracy: 0.9451 - acc: 0.9451 - f1: 0.4618
Epoch 5/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5397 - binary_accuracy: 0.9447 - acc: 0.9447 - f1: 0.4612
Epoch 6/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5405 - binary_accuracy: 0.9451 - acc: 0.9451 - f1: 0.4600
Epoch 7/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5393 - binary_accuracy: 0.9450 - acc: 0.9450 - f1: 0.4613

4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5382 - binary_accuracy: 0.9454 - acc: 0.9454 - f1: 0.4623
Epoch 58/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5383 - binary_accuracy: 0.9450 - acc: 0.9450 - f1: 0.4622
Epoch 59/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5382 - binary_accuracy: 0.9452 - acc: 0.9452 - f1: 0.4626
Epoch 60/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5389 - binary_accuracy: 0.9451 - acc: 0.9451 - f1: 0.4617
Epoch 61/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5389 - binary_accuracy: 0.9451 - acc: 0.9451 - f1: 0.4617
Epoch 62/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5408 - binary_accuracy: 0.9448 - acc: 0.9448 - f1: 0.4601
Epoch 63/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5416 - binary_accuracy: 0.9436 - acc: 0.9436 - f1: 0.4586
Epoch

4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5382 - binary_accuracy: 0.9453 - acc: 0.9453 - f1: 0.4623
Epoch 114/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5380 - binary_accuracy: 0.9453 - acc: 0.9453 - f1: 0.4625
Epoch 115/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5379 - binary_accuracy: 0.9452 - acc: 0.9452 - f1: 0.4626
Epoch 116/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5379 - binary_accuracy: 0.9456 - acc: 0.9456 - f1: 0.4627
Epoch 117/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5378 - binary_accuracy: 0.9453 - acc: 0.9453 - f1: 0.4627
Epoch 118/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5376 - binary_accuracy: 0.9454 - acc: 0.9454 - f1: 0.4628
Epoch 119/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5375 - binary_accuracy: 0.9454 - acc: 0.9454 - f1: 0.4630

4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5483 - binary_accuracy: 0.9424 - acc: 0.9424 - f1: 0.4518
Epoch 170/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5459 - binary_accuracy: 0.9438 - acc: 0.9438 - f1: 0.4543
Epoch 171/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5471 - binary_accuracy: 0.9409 - acc: 0.9409 - f1: 0.4529
Epoch 172/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5458 - binary_accuracy: 0.9454 - acc: 0.9454 - f1: 0.4543
Epoch 173/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5448 - binary_accuracy: 0.9439 - acc: 0.9439 - f1: 0.4553
Epoch 174/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5497 - binary_accuracy: 0.9416 - acc: 0.9416 - f1: 0.4504
Epoch 175/200
4485957/4485957 [==============================] - 2s 0us/step - loss: 0.5443 - binary_accuracy: 0.9436 - acc: 0.9436 - f1: 0.4557

In [43]:
#Predict
y_pred = model.predict(x_test)

In [44]:
#Set to 0s and 1s
y_pred = (y_pred > 0.5)

In [45]:
#Calculate Confusion Matrix
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [46]:
print("spammers accuracy", tp/(fp + tp))
print("non-spammers accuracy", tn/(fn + tn))

spammers accuracy 0.567018349617626
non-spammers accuracy 0.9615290547747624
